In [1]:
from keras.datasets import mnist
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pdb # for debug

Using TensorFlow backend.
/Users/FUZZY/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# ARIYASU code
x_ = np.array([[[[1,2,3,4]],
                [[2,3,4,5]]]]*10) # (10, 2, 1, 4)
w = np.array([[[[1,1,2]],[[2,1,1]]],
                   [[[2,1,1]],[[1,1,1]]],
                   [[[1,1,1]],[[1,1,1]]]])
b = [[[1]],[[2]],[[3]]]
loss = np.array([[[9,11]],
                [[32,35]],
                [[52,56]]])
# 確認用
out_ = np.array([[21,29],
                [18,25],
                [18,24]])
x_delta = np.array([[125,230,204,113],
                    [102,206,195,102]])
w_delta = np.array([[[31,51,71],[51,71,91]],
                    [[102,169,236],[169,236,303]],
                    [[164,272,380],[272,380,488]]])

# 2次元の畳み込みニューラルネットワークスクラッチ

2次元に対応した畳み込みニューラルネットワーク（CNN）のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


プーリング層なども作成することで、CNNの基本形を完成させます。クラスの名前はScratch2dCNNClassifierとしてください。

## データセットの用意
引き続きMNISTデータセットを使用します。2次元畳み込み層へは、28×28の状態で入力します。


今回は白黒画像ですからチャンネルは1つしかありませんが、チャンネル方向の軸は用意しておく必要があります。


`(n_samples, n_channels, height, width)` の`NCHW`または`(n_samples, height, width, n_channels)`の`NHWC`どちらかの形にしてください。



In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
X_train = X_train[:,np.newaxis, :,:, ]
X_test = X_test[:,np.newaxis, :,:, ]

# 画像サイズ縮小処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [186]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

## 【問題1】2次元畳み込み層の作成
1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。


フォワードプロパゲーションの数式は以下のようになります。

$$
a_{i,j,m} = \sum_{k=0}^{K-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1}x_{(i+s),(j+t),k}w_{s,t,k,m}+b_{m}
$$

$a
_{i,j,m}$
 : 出力される配列のi行j列、mチャンネルの値


$i$
 : 配列の行方向のインデックス


$j$
 : 配列の列方向のインデックス


$m$
 : 出力チャンネルのインデックス


$K$
 : 入力チャンネル数


$F_h
,F_w$
 : 高さ方向（$h$）と幅方向（$w$）のフィルタのサイズ


$x_{(i+s),(j+t),k}$
 : 入力の配列の(i+s)行(j+t)列、kチャンネルの値


$w_{s,t,k,m}$
 : 重みの配列のs行t列目。kチャンネルの入力に対して、mチャンネルへ出力する重み


b_m
 : mチャンネルへの出力のバイアス項


全てスカラーです。


次に更新式です。1次元畳み込み層や全結合層と同じ形です。

$$
w_{s,t,k,m}^{\prime} = w_{s,t,k,m} - \alpha \frac{\partial L}{\partial w_{s,t,k,m}} \\
b_{m}^{\prime} = b_{m} - \alpha \frac{\partial L}{\partial b_{m}}
$$

$α$
  : 学習率


$\frac{\partial L}{\partial w_{s,t,k,m}} : 
w_{s,t,k,m}$ に関する損失 $L$の勾配


$\frac{\partial L}{\partial b_{m}}: b_m$
 に関する損失 
$L$
 の勾配


勾配 
$\frac{\partial L}{\partial w_{s,t,k,m}}$
 や $\frac{\partial L}{\partial b_{m}}$
 を求めるためのバックプロパゲーションの数式が以下である。
 
 $$
 \frac{\partial L}{\partial w_{s,t,k,m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1} \frac{\partial L}{\partial a_{i,j,m}}x_{(i+s)(j+t),k}\\
\frac{\partial L}{\partial b_{m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1}\frac{\partial L}{\partial a_{i,j,m}}
 $$
 
$\frac{\partial L}{\partial a_i}$: 勾配の配列のi行j列、mチャンネルの値

$N_{out,h},N_{out,w}$: 高さ方向（h）と幅方向（w）の出力のサイズ

前の層に流す誤差の数式は以下です。

$$
\frac{\partial L}{\partial x_{i,j,k}} = \sum_{m=0}^{M-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1} \frac{\partial L}{\partial a_{(i-s),(j-t),m}}w_{s,t,k,m}
$$

$\frac{\partial L}{\partial x_{i,j,k}}$: 前の層に流す誤差の配列のi列j行、kチャンネルの値

$M$
  : 出力チャンネル数
  
ただし、 
$i−s<0$
 または $i−s>N_{out,h}−1$
 または $j−t<0$
 または $j−t>N_{out,w}−1$
 のとき  
 $\frac{\partial L}{\partial a_{(i-s),(j-t),m}} =0$です。

### Conv2d

In [124]:
class Conv2d:
    """

    """
    def __init__(self, x_shape, FN):
        self.FN = FN
        N, CH, H, W = x_shape # X.shape
        # Padding 
        self.P = 0
        # stride
        self.S = 1 
        # Learning Rate
        self.lr = 1 
        # Weight
        FH, FW = 3, 3 # filter size
        self.W = np.ones((FN, CH, FH, FW))
        #print('W.shape', self.W.shape)

        # Bias
        self.B = np.ones(CH)
        
    def forward(self, x):
        """

        """
        self.x = x # for backward
        N, CH, H, W = x.shape
        FN, CH, FH, FW = self.W.shape
#         pdb.set_trace()
        H_out, W_out = int((H + 2*self.P - FH)/self.S + 1), int((W + 2*self.P - FW)/self.S + 1)
        # Feature Map(N, CH, H_out, W_out)
        self.FM = np.empty((N, FN, H_out, W_out)) 
        #print('FM.shape', self.FM.shape)
        for i in range(N): # loop for n_sample times
            for j in range(FN): # loop for n_filter times
                for k in range(H_out):
                    for l in range(W_out):
#                         pdb.set_trace()
                        #print()
                        self.FM[i, j, k, l] = np.sum(x[i, :, k:k+FH, l:l+FW]  * self.W[j, :, :, :]) + self.B
                        #print(self.FM.shape)
        return self.FM

    
    def backward(self, dA): # dA(20, 32, 26, 26) -> X(20, 1, 28, 28)
        """

        """
#         pdb.set_trace()
        N, CH, H, W = self.x.shape
        FN, CH, FH, FW = self.W.shape
        dZ = np.empty((self.x.shape))
        for i in range(N): # 2→入力ch 
            for j in range(CH): # N_in = ４→Xの特徴量
                for k in range(FH): # 3→w.shape[0]→出力ch
                    for l in range(FW): # Wの数 = 3
                        dZ[i, j, k:k + FH, l:l + FW] += np.sum(dA[i, j, k, l]* self.W[j, :, :, :]) + self.B 

#         pdb.set_trace() 
        self.dZ = dZ           
        self.N_out = int((CH + 2* self.P - FN)/self.S + 1)
        
        # w 更新
        new_W = np.ones(self.W.shape) # new_W.shape (32, 1, 3, 3)
        for i in range(N): # 2→入力ch 
            for j in range(CH): # N_in = ４→Xの特徴量
                for k in range(FH): # 3→w.shape[0]→出力ch
                    for l in range(FW): # Wの数 = 3
                        new_W[i, j, k, l] = np.sum(self.x[i, :, k:k+FH, l:l+FW]*dA[i, j, k, l])
        self.W -= self.lr*new_W
        
        # b 更新  
        new_B = np.ones(self.B.shape)
        for i in range(self.x.shape[1]):
            new_B[i] = np.sum(dA[:,i,:,:])
        self.B -= self.lr*new_B     
    
    
        return dZ

## 【問題2】2次元畳み込み後の出力サイズ
畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。

$$
N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
$$

$N_{out}$
  : 出力のサイズ（特徴量の数）


$N_{in}$
 : 入力のサイズ（特徴量の数）


$P$
 : ある方向へのパディングの数


$F$
 : フィルタのサイズ


$S$
 : ストライドのサイズ


$h$
 が高さ方向、 
$w$
 が幅方向である

In [82]:
def output_func(N_in, P, F, S):
    return int((N_in + 2*P - F)/S + 1)
    
Fh = 3
Fw = 3
Sh = 1
Sw = 1
Ph = 0
Pw = 0

Nh_in = 28
Nw_in = 28

Nh_out = output_func(Nh_in, Ph, Fh, Sh)
Nw_out = output_func(Nw_in, Pw, Fw, Sw)

Nh_out, Nw_out

(26, 26)

## 【問題3】最大プーリング層の作成
最大プーリング層のクラスMaxPool2Dを作成してください。プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。
$$
a_{i,j,k} = \max_{(p,q)\in P_{i,j}}x_{p,q,k}
$$

$P_{i,j}$
 : $i$行$j$列への出力する場合の入力配列のインデックスの集合。 

$S_h
×
S_w$
 の範囲内の行（$p$）と列（$q$）


$S_h,S_w$
 : 高さ方向（$h$）と幅方向（$w$）のストライドのサイズ


$(p,q)∈P_{i,j}: P_{i,j}$
 に含まれる行（$p$）と列（$q$）のインデックス


$a_{i,j,m}$
 : 出力される配列の$i$行$j$列、$k$チャンネルの値


$x_{p,q,k}$
 : 入力の配列の$p$行$q$列、$k$チャンネルの値


ある範囲の中でチャンネル方向の軸は残したまま最大値を計算することになります。


バックプロパゲーションのためには、フォワードプロパゲーションのときの最大値のインデックス 
$(p,q)$
 を保持しておく必要があります。フォワード時に最大値を持っていた箇所にそのままの誤差を流し、そこ以外には0を入れるためです。



### MaxPool2D

In [83]:
#入力はNHWC(n_samples, height, width, n_channels)
class MaxPool2D:
    
    def __init__(self, ch):
        self.FH = 2
        self.FW = 2
        self.S = 1 # stride
        
    def forward(self, x):
        self.A = np.zeros(x.shape)
        
        
        sample = x.shape[0]
        chanel = x.shape[1]
        height = x.shape[2]
        width = x.shape[3]
        self.Z_index = np.zeros([sample,chanel,height,width])
        Z = np.zeros([sample,chanel,int(height/2),int(width/2)])

        for h in range(sample):#サンプルを設定
            for k in range(chanel):#チャネルを設定
                for i in range(self.FH):#高さを設定 フィルタ＝ストライド＝２
                    for j in range(self.FW):#幅を設定
                            Z0 = x[h,k,i*2:i*2 + 2,j*2:j*2 + 2]
                            Z[h,k,i,j] = np.nanmax(Z0)
                            a = Z0/np.nanmax(Z0)#最大値が１の行列
                            self.Z_index[h,k,i*2:i*2 + 2,j*2:j*2 + 2] += (np.where(a == 1 ,1,0)) # z idx keep
        return Z
    
    def backward(self, x):
        return self.A
        

## 【問題5】平滑化
平滑化するためのFlattenクラスを作成してください。


フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。


この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。

### Flatten

In [84]:
class Flatten:
    
    def __init__(self):
        pass
    
    def forward(self, x):
        self.flatten = x.shape
        self.FLT_shape = x.reshape(x.shape[0], -1)[1]
        return x.reshape(x.shape[0], -1)
    
    def backward(self, x):
        return x.reshape(self.flatten)
        
    

# 検証

## 【問題6】学習と推定
作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。


精度は低くともまずは動くことを目指してください。

### Initializer

#### SimpleInitializer

In [85]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)       
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

#### Xavier

In [86]:
class Xavier:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      Xavierの初期値
    """
    def __init__(self, n_nodes1):
        self.sigma = 1/np.sqrt(n_nodes1) # self.batch_sizeかも
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)       
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

#### He

In [87]:
class He:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      Xavierの初期値
    """
    def __init__(self, n_nodes1):
        self.sigma = np.sqrt(2/n_nodes1) 
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)       
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

### Optimizer

In [88]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        # layerの引数は"class FCのインスタンス自体"を取得している。

        layer.B -= self.lr*layer.B_dash # B更新
        layer.W -= self.lr*layer.W_dash # W更新


### FC

In [89]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes_out, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
#         self.n_nodes1 = n_nodes1
        self.n_nodes_out = n_nodes_out
        self.initializer = initializer
        self.optimizer = optimizer
        self.H_W = 0
        self.H_B = 0
        
    def forward(self, x, n_nodes_in):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        
        self.n_nodes_in = n_nodes_in
        self.W = self.initializer.W(self.n_nodes_in, self.n_nodes_out)
        self.B = self.initializer.B(self.n_nodes_out)
        self.Z = x # 前のレイヤーのZを取得
        A = x@self.W + self.B
        
        return A

    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
#         pdb.set_trace()
        dZ = dA@self.W.T # このレイヤーのW

        # 更新
        self.B_dash = np.mean(dA, axis=0)
        self.W_dash = self.Z.T@dA # 前のレイヤーのZ
        self = self.optimizer.update(self) # この引数のselfはこの"class FCのインスタンス自体"を取得している。
        return dZ

### 活性化関数

In [90]:
class ReLU:

    def __init__(self):
        pass
    
    def forward(self, x):
        self.F = np.where(x > 0, 1, 0) # backwardの引数のためのAを保管
        return np.maximum(x, 0)
    
    def backward(self, x):
#         pdb.set_trace()
        return x*self.F
    

class Softmax:
    def __init__(self):
        pass
    def cross_entropy_func(self, x, y):
        li = np.empty(len(x))
        for i in range(len(x)):
            lj = np.empty(10) # 10 = self.n_output, 最終的に変数にする
            for j in range(10): # 10 = self.n_output, 最終的に変数にする
                lj[j] = y[i,j]*np.log(x[i,j])
            li[i] = sum(lj)
        self.l = sum(li)/-10 # 10 = self.n_output, 最終的に変数にする
    
    def forward(self, x):
        x_max = np.max(x, axis=1)
        exp_x = np.exp(x - x_max.reshape(-1, 1))
        sum_exp_x = np.sum(exp_x, axis=1).reshape(-1, 1)  
        return exp_x/sum_exp_x

    def backward(self, x, y):
        self.cross_entropy_func(x, y)
        return x - y

In [91]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=42):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [132]:
class Conv2d_im2col:
    """
    
    """
    def __init__(self, FN, FC, FH, FW, lr, stride=1, padding=0, sigma=0.01):
        self.FN = FN
        self.FC = FC
        self.FH = FH
        self.FW = FW
        self.lr = lr
        self.stride = stride
        self.pad = padding
        self.w = sigma * np.random.randn(self.FN, self.FC, self.FH, self.FW)
        self.b = np.zeros(self.FN)
        
    # im２col利用したforward関数
    def forward(self, X):
        """
        ２次元畳み込みフォアードの関数
        ---------------
        parameter
        ---------------
        X: 入力データ
        N: 入力データの個数
        H: 入力データの高さ
        W:入力データの幅
        i :　配列行方向インデックス
        j:　配列列方向のインデックス
        m:　出力チャネルのインデックス
        C:　入力チャネル数
        FN: フィルタのこすう
        FH:　フィルタの高さ
        FW:　フィルタの幅
        FC:　フィルタのチャネル
        W: フィルタ
        b：　バイアス
        """ 
        self.X = X
        
        #各データのチャネル、高さ、幅の取り出し
        self.N ,self.C, self.H, self.W = X.shape
        self.FN, self.FC, self.FH, self.FW = self.w.shape
        OH, OW = self.N_out_size(X, self.w,  self.pad, self.stride)

        # im2col(x, FH, FW, stride, padding)
        self.col = self.im2col(self.X, self.FH, self.FW, self.stride, self.pad)
        self.col_w = self.w.reshape(self.FN, -1).T
        out = np.dot(self.col, self.col_w) + self.b
        out = out.reshape(self.N, OH, OW, -1).transpose(0, 3, 1, 2)
        return out
        
    # backward関数
    def backward(self, n_out):
        """
        ２次元畳み込みバックプロバゲーションの関数
        ---------------
        parameter
        ---------------
        n_out:　入力データ

        ---------------
        return
        ---------------
        dx: デルタX値
        """
        n_out = n_out.transpose(0, 2, 3, 1).reshape(-1, self.FN)

        self.db = np.sum(n_out, axis=0)
        self.dw = np.dot(self.col.T, n_out)
        self.dw = self.dw.transpose(1, 0).reshape(self.FN, self.FC, self.FH, self.FW)

        dcol = np.dot(n_out, self.col_w.T)
        self.dx = self.col2im(dcol, self.X.shape, self.FH, self.FW, self.stride, self.pad)
        self.w, self.b = self._update()
        return self.dx
    
    # update関数
    def _update(self):
        """
        フィルタとバイアス更新式
        ----------------
        parameter
        ----------------
        lr: 学習率
        w：フィルタ
        b：バイアス
        ---------------
        return
        ---------------
        w ： 更新後のW 
        b ：　更新後のb
        """
        self.w = self.w  - self.lr * self.dw
        self.b = self.b -self. lr * self.db

        return  self.w, self.b 
    
    def N_out_size(self,N, F, P, S):
        """
        出力データのサイズを導く関数
        -------------------
        parameter
        -------------------
        N:　入力データ
        Nh：入力データの高さ
        Nw：　入力データの幅
        F：　filter
        Fh：フィルタの高さ
        Fw：　フィルタの幅
        P：　パディング
        S: ストライド
        Sh：ストライドの高さ方向
        Sw：ストライドの幅方向
        ------------------
        return
        ------------------
        Nh_out：出力データの高さ
        Nw_out：　出力データの幅
        """
        _, _, NH, NW = N.shape
        _, _, FH , FW = F.shape

        NH_out = (NH + 2*P - FH) // S +1
        NW_out = (NW + 2*P - FW) // S +1 
        return NH_out,  NW_out
        
    # im2col関数
    def im2col(self, input_data, FH, FW, stride, pad):
        N, C, H, W = input_data.shape
        #出力データのサイズ
        out_h = (H + 2*pad - FH)//stride + 1
        out_w = (W + 2*pad - FW)//stride + 1

        # paddingの追加
        img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
        # データの空箱作成
        col = np.zeros((N, C, FH, FW, out_h, out_w))

        for y in range(FH):
            y_max = y + stride*out_h
            for x in range(FW):
                x_max = x + stride*out_w
                col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
        return col      

    #col2im
    def col2im(self,col, input_shape, FH, FW, stride=1, pad=0):
        N, C, H, W = input_shape
        #出力データのサイズ
        out_h = (H + 2*pad - FH)//stride + 1
        out_w = (W + 2*pad - FW)//stride + 1
        col = col.reshape(N, out_h, out_w, C, FH, FW).transpose(0, 3, 4, 5, 1, 2)

        img = np.zeros((N, C, H + 2*pad + stride -1, W+ 2*pad+stride-1))
        for y in range(FH):
            y_max = y + stride * out_h
            for x in range(FW):
                x_max = x + stride * out_w
                img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

        return img[:, :, pad:H + pad, pad:W + pad]

### ScratchCNNClassifier

In [172]:
# class ScratchCNNClassifier:

#     def __init__(self, verbose=None, batch_size=20, a=0.01, n_epoch=1):
#         self.verbose = verbose
#         self.batch_size = batch_size
#         self.lr = a
#         self.n_epoch = n_epoch


#     def fit(self, X, y, X_val=None, y_val=None):
# # 初期値
#         NODE_TO_SOFTMAX = 10 # softmaxに送るノード数（固定）
#         self.sigma = 0.01 # ガウス分布の標準偏差
#         self.x = X
        
# # X設定　（60000, 1, 28, 28）の場合想定        
# #         self.n_samples, self.input_ch, self.height, self.width = X.shape        
#         self.n_samples = X.shape[0]        
#         self.input_ch = X.shape[1]        
#         self.height = X.shape[2]        
#         self.width = X.shape[3]        
        
# # Bias
#         self.B_1 = X.shape[1] # Bのシェイプ（output_ch）
        
# # fitメソッド内        
#         self.Conv2d1 = Conv2d_im2col(FN=32, FC=1, FH=3, FW=3, lr = self.lr)
#         self.activation1 = ReLU()
#         self.MaxPool2D_1 = MaxPool2D(20)
#         self.Flatten = Flatten()
#         optimizer = SGD(self.lr)
#         self.FC1 = FC(NODE_TO_SOFTMAX, SimpleInitializer(self.sigma), optimizer)
#         self.SoftMax = Softmax()
        
#         get_mini_batch = GetMiniBatch(X, y, batch_size = self.batch_size)
#         self.l_list = np.empty(self.n_epoch)
#         self.l_val_list = np.empty(self.n_epoch)
#         self.cnt = 0
#         self.cnt_list = []
#         CNT = 0
#         for i in range(self.n_epoch):
#             for X, y in get_mini_batch:        
        
        
# # イテレーションごとのフォワード  
#                 # Input Image (20, 1, 28, 28)
#                 # print(X.shape)
#                 FM1 = self.Conv2d1.forward(X) # (20, 32, 26, 26)
#                 # print(FM1.shape)
#                 Z1 = self.activation1.forward(FM1) # (20, 32, 26, 26)
#                 # print(Z1.shape)
#                 FM2 = self.MaxPool2D_1.forward(Z1) # (20, 32, 13, 13)
#                 # print(FM2.shape)
#                 FLT = self.Flatten.forward(FM2) # (20, 5408)
#                 # print(FLT.shape)
#                 n_node_in = FLT.shape[1]
#                 A1 = self.FC1.forward(FLT, n_node_in) # (20, 10) 
#                 # print(A1.shape)
#                 Z2 = self.SoftMax.forward(A1) # (20, 10) 
#                 # print(Z2.shape)
#                 # pdb.set_trace()
# # イテレーションごとのバックワード
#                 dA1 = self.SoftMax.backward(Z2, y) # (20, 10) 
#                 # print(dA1.shape)
#                 # pdb.set_trace()
#                 dFLT = self.FC1.backward(dA1) # (20, 5408)
#                 # print(dFLT.shape)
#                 dFM2 = self.Flatten.backward(dFLT) # (20, 32, 13, 13)
#                 # print(dFM2.shape)
#                 dZ1 = self.MaxPool2D_1.backward(dFM2) # (20, 32, 26, 26)
#                 # print(dZ1.shape)
#                 dFM1 = self.activation1.backward(dZ1) # (20, 32, 26, 26)
#                 # print(dFM1.shape)
#                 dZ0 = self.Conv2d1.backward(dFM1) # (20, 1, 28, 28) ＊dZ0は使用しない        
                
# #                 print(CNT)
# #                 CNT+=1

#     def predict(self, X):
#         """
#         ニューラルネットワーク分類器を使い推定する。

#         Parameters
#         ----------
#         X : 次の形のndarray, shape (n_samples, n_features)
#             サンプル

#         Returns
#         -------
#             次の形のndarray, shape (n_samples, 1)
#             推定結果
#         """
#         A1 = self.Conv2d1.forward(X) # (26, 26, 32)
#         A2 = self.activation1.forward(A1) # (13, 13, 32)
#         A3 = self.MaxPool2D_1.forward(A2) # (11, 11, 64)
#         A4 = self.Flatten.forward(A3) # (5, 5, 64)
#         n_node_in = A4.shape[1]
#         A5 = self.FC1.forward(A4, n_node_in) # (3, 3, 128)
#         A6 = self.SoftMax.forward(A5) # (1, 1, 128)
#         result = np.argmax(A6, axis=1)
#         print(A6[0:10])
#         return result

In [198]:
# スクラッチのCNN
class ScratchCNNClassifier2:
    """
    n_iter :学習回数
    lr: 学習率
    sigma:初期値
    batch:バッチサイズ
    loss:lossの保管
    val_loss:バリデーションのloss保管
    filter_output:アウトプットの値
    filter_channel:フィルタのチャネル数
    filter_size:フィルタのサイズ
    """
    def __init__(self, lr=0.01,sigma=0.1, n_iter=3, batch=20, filter_output=30, filter_channel=1, filter_size=3, vervose=False):
        # 　学習回数
        self.iter = n_iter
        self.lr = lr
        self.sigma = sigma
        self.batch = batch
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        self.filter_output = filter_output
        self.filter_channel = filter_channel
        self.filter_size = filter_size
        
    
    def fit(self, X, y,  X_val=None, y_val=None):
        self.n_features = X.shape[0]
        self.n_output = 10
        # 入力用のデータ
        input_size = 28
        padding = 0
        stride = 1
        filter_num = 30
        conv_output_size = (input_size - self.filter_size + 2 * padding) / stride + 1
        pool_output_size = int(self.filter_output * (conv_output_size / 2) * (conv_output_size / 2))


        #更新方法の定義
        self.optimizer = AdaGrad(self.lr, self.batch)
        #　畳み込み層の定義
        self.Conv2d = Conv2d_im2col(FN=30, FC=1, FH=3, FW=3, lr = self.lr)
        # １層目出力データの活性化の定義
        self.Conv_activation = Relu()
        # pooling処理の定義
        self.Pool = Maxpool2D_im2col()
        #　平滑化処理の定義
        self.flatten = Flatten()
        # 全結合層の定義
        self.FC1 = FC(18750, self.n_output, HeInitializer(self.filter_output), self.optimizer)

        # 最終層の活性化の定義
        self.activation1 = Softmax()

        # 学習開始
        for i in range(self.iter):
            #  バッチサイズのカウント
            count = 0
            history = np.zeros(X.shape[0] // self.batch)
            if X_val:
                val_history = np.zeros(X_val.shape[0] // self.batch)
            #　ミニバッチの取り出しコード
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch)
            for mini_X_train, mini_y_train in get_mini_batch:
                # forward
                A0 = self.Conv2d.forward(mini_X_train)#畳み込み層
                Z0 = self.Conv_activation.forward(A0)#RELUで活性化
                P0 = self.Pool.forward(Z0)#pooling 
                F0 = self.flatten.forward(P0)#flattenして平滑化
                A1 = self.FC1.forward(F0)#全結合層
                Z1 = self.activation1.forward(A1)#Softmaxで活性化

                # backward
                dA1, history[count] = self.activation1.backward(Z1, mini_y_train)#Softmaxのバック
                #flatten元の形に戻す
                dZ1 = self.FC1.backward(dA1)#全結合層のバック
                #flatten元の形に戻す
                dF0 = self.flatten.backward(dZ1)
                #pooling層のバック
                dP0 = self.Pool.backward(dF0)
                dA0 = self.Conv_activation.backward(dP0)#RELUのバック
                dZ0 = self.Conv2d.backward(dA0)#畳み込み層のバック
                count += 1
            
            #loss保管のため学習したパラメータでforward
            A0 = self.Conv2d.forward(X)#畳み込み層
            Z0 = self.Conv_activation.forward(A0)#RELUで活性化
            P0 = self.Pool.forward(Z0)#pooling 
            F0 = self.flatten.forward(P0)#flattenして平滑化
            A1 = self.FC1.forward(F0)#全結合層
            Z1 = self.activation1.forward(A1)#Softmaxで活性化
            
            _, history= self.activation1.backward(Z1, y)
            self.loss[i] = history
            
            # X_valあった場合の処理
            if np.any(X_val): 
                A0 = self.Conv2d.forward(X_val)
                Z0 = self.Conv_activation.forward(A0)
                P0 = self.Pool.forward(Z0)#pooling 
                F0 = self.flatten.forward(P0)#flattenして平滑化
                A1 = self.FC1.forward(F0)
                Z1 = self.activation1.forward(A1)
                # 学習記録
                _, val_history = self.activation1.backward(Z1, y_val)
                self.val_loss[i] = val_history
            
            
    # 推定
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        print(X.shape)
        A0 = self.Conv2d.forward(X)#畳み込み層
        Z0 = self.Conv_activation.forward(A0)#RELUで活性化
        P0 = self.Pool.forward(Z0)#pooling 
        F0 = self.flatten.forward(P0)#flattenして平滑化
        A1 = self.FC1.forward(F0)#全結合層
        Z1 = self.activation1.forward(A1)#Softmaxで活性化
        return np.argmax(Z1, axis= 1)
    
    
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.w = initializer.W(self.n_nodes1, self.n_nodes2)
        self.b =  initializer.B(self.n_nodes2)
        # adagrad用
        #if Adagrad = True: 
        self.h_w = np.zeros((n_nodes1, n_nodes2))
        self.h_b = np.zeros(n_nodes2)

    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """     
        self.Z = X
        
        self.A =  (X @ self.w) + self.b
        return self.A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.db = np.sum(dA, axis=0)
        self.dw = self.Z.T @ dA 
        dZ = dA @ self.w.T
        # 更新
        self = self.optimizer.update(self)
        return dZ    

    
#初期化
class HeInitializer():
    
    def __init__(self, sigma):
        self.sigma = np.sqrt(2 / sigma)
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :　重みの初期値
        """
        self.w = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return self.w
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :　バイアスの初期値
        """
        self.b = self.sigma * np.random.randn(n_nodes2)
        return self.b

    
# 活性化関数
class Softmax():
    def forward(self, A):
        # overflow対策
        A_max = np.max(A, axis=1)
        exp_A = np.exp(A - A_max.reshape(-1, 1))
        sum_exp_A = np.sum(exp_A, axis=1).reshape(-1, 1)
        return exp_A / sum_exp_A
    
    def backward(self, Z, Y):
        loss = self._cross_entropy(Z, Y)
        D  = Z - Y
        return D, loss 
    
    def _cross_entropy(self,Z, y):
        """
        パラメータ
        ーーーーーーーーーーーーー
        y : 正解ラベル（one-hot表現）
        z３　：　クラスの確率
        n : バッチサイズ

        """
        if y.ndim == 1:    # 次元が 1 の場合
            y = y.reshape(1, y.size)
        batch_size = y.shape[0]
        return -1* np.sum(y * np.log(Z)) / batch_size


# 活性化関数
class Relu():
    def forward(self, A):
        self.mask = (A <= 0)
        a = A.copy()
        self.A = A
        a[self.mask] = 0
        return np.maximum(0, A)
      
    def backward(self, dA):
        dA[self.mask] = 0
        out = dA
        return out

    
# AdaGrad
class AdaGrad:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr, batch):
        self.lr = lr
        self.batch = batch
        # 過去の勾配の２乗和の保管用
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.h_w += (layer.dw/ self.batch) * (layer.dw/ self.batch)
        layer.h_b += (layer.db/self.batch) * (layer.db / self.batch)
        layer.w -= self.lr * (1 / (np.sqrt(layer.h_w) + 1e-07)) * (layer.dw/ self.batch)
        layer.b -= self.lr * (1 / (np.sqrt(layer.h_b) + 1e-07)) * (layer.db/ self.batch)
        
#         layer.h_W += layer.dW * layer.dW
#         layer.h_B += layer.dB * layer.dB
#         layer.W -= self.lr * (1 / (np.sqrt(layer.h_W) + 1e-07)) * layer.dW
#         layer.B -= self.lr * (1 / (np.sqrt(layer.h_B) + 1e-07)) * layer.dB
        return layer
                                 
    
#　畳み込み層  
class Conv2d_im2col:
    """
    
    """
    def __init__(self, FN, FC, FH, FW, lr, stride=1, padding=0, sigma=0.01):
        self.FN = FN
        self.FC = FC
        self.FH = FH
        self.FW = FW
        self.lr = lr
        self.stride = stride
        self.pad = padding
        self.w = sigma * np.random.randn(self.FN, self.FC, self.FH, self.FW)
        self.b = np.zeros(self.FN)
        
    # im２col利用したforward関数
    def forward(self, X):
        """
        ２次元畳み込みフォアードの関数
        ---------------
        parameter
        ---------------
        X: 入力データ
        N: 入力データの個数
        H: 入力データの高さ
        W:入力データの幅
        i :　配列行方向インデックス
        j:　配列列方向のインデックス
        m:　出力チャネルのインデックス
        C:　入力チャネル数
        FN: フィルタのこすう
        FH:　フィルタの高さ
        FW:　フィルタの幅
        FC:　フィルタのチャネル
        W: フィルタ
        b：　バイアス
        """ 
        self.X = X
        
        #各データのチャネル、高さ、幅の取り出し
        self.N ,self.C, self.H, self.W = X.shape
        self.FN, self.FC, self.FH, self.FW = self.w.shape
        OH, OW = self.N_out_size(X, self.w,  self.pad, self.stride)

        # im2col(x, FH, FW, stride, padding)
        self.col = self.im2col(self.X, self.FH, self.FW, self.stride, self.pad)
        self.col_w = self.w.reshape(self.FN, -1).T
        out = np.dot(self.col, self.col_w) + self.b
        out = out.reshape(self.N, OH, OW, -1).transpose(0, 3, 1, 2)
        return out
        
    # backward関数
    def backward(self, n_out):
        """
        ２次元畳み込みバックプロバゲーションの関数
        ---------------
        parameter
        ---------------
        n_out:　入力データ

        ---------------
        return
        ---------------
        dx: デルタX値
        """
        n_out = n_out.transpose(0, 2, 3, 1).reshape(-1, self.FN)

        self.db = np.sum(n_out, axis=0)
        self.dw = np.dot(self.col.T, n_out)
        self.dw = self.dw.transpose(1, 0).reshape(self.FN, self.FC, self.FH, self.FW)

        dcol = np.dot(n_out, self.col_w.T)
        self.dx = self.col2im(dcol, self.X.shape, self.FH, self.FW, self.stride, self.pad)
        self.w, self.b = self._update()
        return self.dx
    
    # update関数
    def _update(self):
        """
        フィルタとバイアス更新式
        ----------------
        parameter
        ----------------
        lr: 学習率
        w：フィルタ
        b：バイアス
        ---------------
        return
        ---------------
        w ： 更新後のW 
        b ：　更新後のb
        """
        self.w = self.w  - self.lr * self.dw
        self.b = self.b -self. lr * self.db

        return  self.w, self.b 
    
    def N_out_size(self,N, F, P, S):
        """
        出力データのサイズを導く関数
        -------------------
        parameter
        -------------------
        N:　入力データ
        Nh：入力データの高さ
        Nw：　入力データの幅
        F：　filter
        Fh：フィルタの高さ
        Fw：　フィルタの幅
        P：　パディング
        S: ストライド
        Sh：ストライドの高さ方向
        Sw：ストライドの幅方向
        ------------------
        return
        ------------------
        Nh_out：出力データの高さ
        Nw_out：　出力データの幅
        """
        _, _, NH, NW = N.shape
        _, _, FH , FW = F.shape

        NH_out = (NH + 2*P - FH) // S +1
        NW_out = (NW + 2*P - FW) // S +1 
        return NH_out,  NW_out
        
    # im2col関数
    def im2col(self, input_data, FH, FW, stride, pad):
        N, C, H, W = input_data.shape
        #出力データのサイズ
        out_h = (H + 2*pad - FH)//stride + 1
        out_w = (W + 2*pad - FW)//stride + 1

        # paddingの追加
        img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
        # データの空箱作成
        col = np.zeros((N, C, FH, FW, out_h, out_w))

        for y in range(FH):
            y_max = y + stride*out_h
            for x in range(FW):
                x_max = x + stride*out_w
                col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
        return col      

    #col2im
    def col2im(self,col, input_shape, FH, FW, stride=1, pad=0):
        N, C, H, W = input_shape
        #出力データのサイズ
        out_h = (H + 2*pad - FH)//stride + 1
        out_w = (W + 2*pad - FW)//stride + 1
        col = col.reshape(N, out_h, out_w, C, FH, FW).transpose(0, 3, 4, 5, 1, 2)

        img = np.zeros((N, C, H + 2*pad + stride -1, W+ 2*pad+stride-1))
        for y in range(FH):
            y_max = y + stride * out_h
            for x in range(FW):
                x_max = x + stride * out_w
                img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

        return img[:, :, pad:H + pad, pad:W + pad]
    
    
# im2col版Maxpooling
class Maxpool2D_im2col:
    def __init__(self, pool_size=(2,2), stride=1, padding=0):
        self.PH,  self.PW = pool_size
        self.stride = stride
        self.pad = padding
        
    def forward(self, X):
        """
        parameter
        --------------
        N：入力データのこすう
        C：入力データのチャネル数
        H：入力データの高さ
        W：入力データの幅
        
        return
        --------------
        out : maxpooling
        
        """
        self.X_shape = X.shape
        self.N, self.C, self.H, self.W = self.X_shape
        out_h = (self.H + 2*self.pad - self.PH) // self.stride + 1
        out_w = (self.W + 2*self.pad - self.PW)// self.stride + 1
        col = self.im2col(X, self.PH, self.PW, self.stride, self.pad)
        col = col.reshape(-1, self.PH*self.PW)
        
        
        index_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(self.N, out_h, out_w, self.C).transpose(0, 3, 1, 2)
        self.index_max = index_max
        return out 
    
    def backward(self, n_out):
        """
        """
        n_out = n_out.transpose(0, 2, 3, 1)
        
        pool_size = self.PH * self.PW
        out_max = np.zeros((n_out.size, pool_size))
        out_max[np.arange(self.index_max.size), self.index_max.flatten()] = n_out.flatten()
        out_max = out_max.reshape(n_out.shape + (pool_size,))
        
        dcol = out_max.reshape(out_max.shape[0] * out_max.shape[1] * out_max.shape[2], -1)
        dx = self.col2im(dcol, self.X_shape, self.PH, self.PW, self.stride, self.pad)
        
        return dx
    
    # im2col関数
    def im2col(self, input_data, FH, FW, stride, pad):
        N, C, H, W = input_data.shape
        #出力データのサイズ
        out_h = (H + 2*pad - FH)//stride + 1
        out_w = (W + 2*pad - FW)//stride + 1

        # paddingの追加
        img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
        # データの空箱作成
        col = np.zeros((N, C, FH, FW, out_h, out_w))

        for y in range(FH):
            y_max = y + stride*out_h
            for x in range(FW):
                x_max = x + stride*out_w
                col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
        return col  
    
    #col2im
    def col2im(self, col, input_shape, FH, FW, stride=1, pad=0):
        N, C, H, W = input_shape
        #出力データのサイズ
        out_h = (H + 2*pad - FH)//stride + 1
        out_w = (W + 2*pad - FW)//stride + 1
        col = col.reshape(N, out_h, out_w, C, FH, FW).transpose(0, 3, 4, 5, 1, 2)

        img = np.zeros((N, C, H + 2*pad + stride -1, W+ 2*pad+stride-1))
        for y in range(FH):
            y_max = y + stride * out_h
            for x in range(FW):
                x_max = x + stride * out_w
                img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

        return img[:, :, pad:H + pad, pad:W + pad]
    
    
# 平滑化   
class Flatten:
    """
    平滑化クラス    
    """
    def forward(self, x):
        """
        Parameters
        ----------
        x : 入力データ
        
        Returns
        ----------
        a : 出力データ
        
        """
        # 出力チャンネル数と画像サイズに該当する次元を減らす
        self.x_shape = x.shape
        
        x = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        
        return x
    
    def backward(self, da):
        """
        Parameters
        ----------
        da : 後ろから流れてきた勾配
        
        Returns
        ----------
        dz : 前に流す勾配
        
        """
        # プーリング層へ流すために、forward時の元の次元数に戻す
        dz = da.reshape(self.x_shape)
        
        return dz

In [199]:
model = ScratchCNNClassifier2()
model.fit(X_train, y_train)

In [200]:
y_pred = model.predict(X_test)

(10000, 1, 28, 28)


In [201]:
y_pred.shape
y_true = np.argmax(y_test, axis=1)
print(y_pred)
print(y_true)
accuracy_score(y_true, y_pred)

[1 1 1 ... 1 1 1]
[7 2 1 ... 4 5 6]


0.1135

## 【問題9】出力サイズとパラメータ数の計算
CNNモデルを構築する際には、全結合層に入力する段階で特徴量がいくつになっているかを事前に計算する必要があります。


また、巨大なモデルを扱うようになると、メモリや計算速度の関係でパラメータ数の計算は必須になってきます。フレームワークでは各層のパラメータ数を表示させることが可能ですが、意味を理解していなくては適切な調整が行えません。


以下の3つの畳み込み層の出力サイズとパラメータ数を計算してください。パラメータ数についてはバイアス項も考えてください。


1.


* 入力サイズ : 144×144, 3チャンネル
* フィルタサイズ : 3×3, 6チャンネル
* ストライド : 1
* パディング : なし

2.


* 入力サイズ : 60×60, 24チャンネル
* フィルタサイズ : 3×3, 48チャンネル
* ストライド　: 1
* パディング : なし

3.


* 入力サイズ : 20×20, 10チャンネル
* フィルタサイズ: 3×3, 20チャンネル
* ストライド : 2
* パディング : なし

＊最後の例は丁度良く畳み込みをすることができない場合です。フレームワークでは余ったピクセルを見ないという処理が行われることがあるので、その場合を考えて計算してください。端が欠けてしまうので、こういった設定は好ましくないという例です。